<a href="https://colab.research.google.com/github/seunomonije/quantum/blob/master/Quantum_Yale_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IBM Qiskit Setup and Examples


### Dependencies

Qiskit installation for the colab environment. Run this line if this is your first time here. 

You can also check if qiskit is already installed by running line 2.

In [ ]:
pip install qiskit

This checks if qiskit was installed correctly and returns the version

*-- SO: As of 8.10.2020 -> version 0.14.2*

In [2]:
import qiskit
qiskit.__version__

'0.14.2'

*-- SO: 
  Do y'all want to run simulations on an actual computer, or would the simulator work for now? Either is fine, I'd just have to sign up for the IBM Quantum Experience and get an API key for us to run any acutal simulations. IMO sounds like something for later, I'm guessing we won't need that within the next week.*

  *If you do want to run on an actual computer let me know and I can set that up ASAP.*

### Use

*-- SO: I include suggestions for code structure here. Please let me know of any improvements I can make in the code design for readability/efficiency.*

<ins>Now we will run a simple circuit</ins>. We achieve this with a few simple steps.


1.   Figure out how we are going to run the circuit. Are we going to use a simulator? Or are we going to try and run it on an actual quantum computer? In this case, we opt for BasicAer's simulator.
2.   Define which registers we will be using. *Why do we need this?*
  * Classical registers are needed in order to look at the outputs of our computations.
  * Since we're using a simulator, we need to "take measurements" one by one and therefore need a place to store an earlier result, since each gate will depend on the results from prior gates.

3.   Build the circuit.
4.   Execute the circuit.







I opted to create a function to measure a qubit here which we we can use later.

In [30]:
# Retrieves the simulator
simulator = qiskit.BasicAer.get_backend('qasm_simulator')

'''
  measureASingleQubit(quantum_register, classical_register)
    creates a quantum circuit which measures a single qubit

    PARAMETERS:
      quantum_register -> the quantum register to use
      classical_register -> the classical register to use

    RETURNS: 
      nothing
'''
def measureASingleQubit(quantum_register, classical_register):

  # create the circuit
  circuit = qiskit.QuantumCircuit(quantum_register, classical_register)

  # add the measurement operation to the circuit
  circuit.measure(quantum_register, classical_register)

  # now let's execute the circuit
  simulation = qiskit.execute(circuit, simulator)
  resultObject = simulation.result()
  
  # get the counts for each qubit
  counts = resultObject.get_counts(circuit)

  # Print out circuit and result for visualization purposes
  print("Circuit: ")
  print(circuit)
  print("Result of simulation:")
  print(counts)

  return 



Links to unobvious documentation:
* [Result Object](https://qiskit.org/documentation/stubs/qiskit.result.Result.html)
  * [Result.get_counts](https://qiskit.org/documentation/stubs/qiskit.result.Result.get_counts.html)



Run the snippet below to see a diagram of the circuit defined in `measureASingleQubit()` as well as the result.



In [ ]:
# Define the registers
qReg = qiskit.QuantumRegister(1);
cReg = qiskit.ClassicalRegister(1);

measureASingleQubit(qReg, cReg)

<ins>Code Refactor</ins>

For the purposes of our work, we will use create a simulator class. The simulator will be global and only allowed to be instantiated once. We can add more and more methods as we expand as well. I would also like to add typing eventually so we know exactly what's going through these functions.

*-- SO: would love feedback*

In [55]:
class Simulator:
    simulator = None

    # Singleton paradigm ensures class is only instantiated once
    __instance = None
    @staticmethod
    def getInstance():
      if Simulator.__instance == None:
        Simulator()
      return Simulator.__instance

    def __init__(self):
      if Simulator.__instance != None:
        raise Exception("The simulator can only be instantiated once.")
      else:
        Simulator.__instance = self
    
    def set(self, simulator):
      self.simulator = simulator
      return
    
    def getCircuitResult(self, circuit):
      simulation = qiskit.execute(circuit, simulator)
      resultObject = simulation.result()
      return resultObject

    # returns counts for each qubit as a dictionary
    def getCircuitResultWithCounts(self, circuit):
      simulation = qiskit.execute(circuit, simulator)
      resultObject = simulation.result()
      counts = resultObject.get_counts(circuit)
      return counts
    
''' EXAMPLE USE '''

# Create a simulator and set what we want it to be
Simulator = Simulator()
Simulator.set(qiskit.BasicAer.get_backend('qasm_simulator'))

# Define the registers
qReg = qiskit.QuantumRegister(1);
cReg = qiskit.ClassicalRegister(1);

# Build a circuit
circuit = qiskit.QuantumCircuit(qReg, cReg)
circuit.measure(qReg, cReg)

# Get the results
print(Simulator.getCircuitResult(circuit))
print(Simulator.getCircuitResultWithCounts(circuit))

Result(backend_name='qasm_simulator', backend_version='2.0.0', header=Obj(backend_name='qasm_simulator', backend_version='2.0.0'), job_id='8eb4cc05-9cc2-4a7b-aac1-60bcd30fdf6c', qobj_id='fdfa66a9-5c34-4443-a05c-f59028cbf7e0', results=[ExperimentResult(data=ExperimentResultData(counts=Obj(0x0=1024)), header=Obj(clbit_labels=[['c23', 0]], creg_sizes=[['c23', 1]], memory_slots=1, n_qubits=1, name='circuit50', qreg_sizes=[['q23', 1]], qubit_labels=[['q23', 0]]), meas_level=<MeasLevel.CLASSIFIED: 2>, name='circuit50', seed_simulator=1374763895, shots=1024, status='DONE', success=True, time_taken=0.0032036304473876953)], status='COMPLETED', success=True, time_taken=0.0032210350036621094)
{'0': 1024}


### Examples